In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parents[1]

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("qepc in root?", (PROJECT_ROOT / "qepc").exists())


In [ ]:
from qepc.brain.games_loader import fetch_league_games, build_games_table

# 👇 Change this line to "2025-26" when you want to run on the current season
season = "2025-26"

team_games = fetch_league_games(season)
games_df = build_games_table(team_games)

print("Season:", season)
print("games_df rows:", len(games_df))
display(games_df.head())


In [ ]:
from qepc.brain.scripts import label_game_scripts_by_total_points

scripts_df = label_game_scripts_by_total_points(
    games_df,
    low_quantile=0.25,
    high_quantile=0.75,
)

print("scripts_df shape:", scripts_df.shape)
display(scripts_df.head())


In [ ]:
scripts_df["SCRIPT_LABEL"].value_counts(normalize=True)


In [ ]:
print("Lowest scoring games (GRIND candidates):")
display(scripts_df.sort_values("TOTAL_POINTS").head(10))

print("Highest scoring games (CHAOS candidates):")
display(scripts_df.sort_values("TOTAL_POINTS", ascending=False).head(10))


In [ ]:
from qepc.brain.boxscores_loader import fetch_boxscores_for_games
from qepc.brain.state_vectors import build_rolling_player_state_vectors

# Use a subset of games to keep things fast for now
game_ids = games_df["GAME_ID"].unique().tolist()[:100]

print("Using", len(game_ids), "games for rolling state sample.")

trad_df, _ = fetch_boxscores_for_games(
    game_ids,
    sleep_seconds=0.7,
    verbose=False,
)

print("trad_df shape:", trad_df.shape)
display(trad_df.head())

rolling_state_df = build_rolling_player_state_vectors(
    boxscores_df=trad_df,
    games_df=games_df,
    window=10,
)

print("rolling_state_df shape:", rolling_state_df.shape)
display(rolling_state_df.head())


In [ ]:
rolling_with_scripts = rolling_state_df.merge(
    scripts_df[["GAME_ID", "SCRIPT_LABEL", "SCRIPT_INDEX"]],
    on="GAME_ID",
    how="left",
)

print("rolling_with_scripts shape:", rolling_with_scripts.shape)
display(rolling_with_scripts.head())


In [ ]:
out_dir = PROJECT_ROOT / "data" / "processed" / "nba" / "scripts"
out_dir.mkdir(parents=True, exist_ok=True)

scripts_df.to_parquet(
    out_dir / f"game_scripts_{season}_by_total_points.parquet",
    index=False,
)

rolling_with_scripts.to_parquet(
    out_dir / f"player_rolling_with_scripts_{season}_sample.parquet",
    index=False,
)

print("Saved scripts + merged rolling_with_scripts to:", out_dir)
